In [13]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
import plotly.express as px
from collections import defaultdict
import numpy as np

from precycle.budget import RenyiBudget
from precycle.budget.curves import GaussianCurve, BoundedOneShotSVT, PureDPtoRDP_loose, PureDPtoRDP, LaplaceCurve
from precycle.utils.plot import df_normalized_curves

In [15]:
epsilon=1.0
print(PureDPtoRDP(epsilon))
print(LaplaceCurve(laplace_noise=1/epsilon))

Budget({1.5: 0.6273326470494991, 1.75: 0.6875040623657688, 2: 0.7353256640555195, 2.5: 0.8032588269330581, 3: 0.8467268304854477, 4: 0.8958832596451837, 5: 0.921715428667875, 6: 0.937351002808619, 8: 0.9552483740548644, 16: 0.9791158874987874, 32: 0.9898947842736058, 64: 0.9950275922616154})
Budget({1.5: 0.5128835112945087, 1.75: 0.5705329201598421, 2: 0.6191236299985928, 2.5: 0.6939505397222643, 3: 0.7468281410689698, 4: 0.8136892965926219, 5: 0.8530780145169694, 6: 0.8787756228833642, 8: 0.9101988011774458, 16: 0.9559067678503111, 32: 0.9781484250454257, 64: 0.9891221586809694})


In [16]:
curves = {}
for epsilon in [0.01, 0.1, 0.5, 1, 5]:
    curves[f"Probabilistic_{epsilon}"] = (PureDPtoRDP(epsilon=3*epsilon) + LaplaceCurve(laplace_noise=1/epsilon))
    curves[f"Deterministic_{epsilon}"] = LaplaceCurve(laplace_noise=1/epsilon)
df = df_normalized_curves(curves)
df["epsilon"] = df.mech_name.apply(lambda s: s.split("_")[1])
df["cache"] = df.mech_name.apply(lambda s: s.split("_")[0])
px.area(df, range_y=[1e-6,1], log_y=False, x="alpha", y="normalized_rdp_epsilon", facet_col="cache", line_group="mech_name", facet_row="epsilon")

In [17]:
curves = {}
for epsilon in [1e-4, 1e-3, 1e-2, 1e-1]:
    curves[f"Probabilistic_{epsilon}"] = (PureDPtoRDP(epsilon=3*epsilon) + LaplaceCurve(laplace_noise=1/epsilon))
    curves[f"Deterministic_{epsilon}"] = LaplaceCurve(laplace_noise=1/epsilon)
df = df_normalized_curves(curves)
df["epsilon"] = df.mech_name.apply(lambda s: s.split("_")[1])
df["cache"] = df.mech_name.apply(lambda s: s.split("_")[0])
px.area(df, range_y=[1e-9,1], log_y=True, x="alpha", y="normalized_rdp_epsilon", facet_col="cache", line_group="mech_name", facet_row="epsilon")

In [55]:
block_epsilon = 10
block_delta = 1e-7
d = []
for epsilon in [1e-5, 1e-4, 1e-3, 1e-2, 0.02, 0.05, 0.07, 1e-1, 0.2, 0.3, 0.5, 1, 2]:
    for pmw_fraction in [0, 0.22, 0.5, 0.75, 1]:
        curve = (PureDPtoRDP(epsilon=3*epsilon) + LaplaceCurve(laplace_noise=1/epsilon)) * pmw_fraction + LaplaceCurve(laplace_noise=1/epsilon) * (1-pmw_fraction)
        block = RenyiBudget.from_epsilon_delta(epsilon=block_epsilon, delta=block_delta)
        normalized_curve = curve.normalize_by(block)
        best_alpha = normalized_curve.alphas[np.argmin(normalized_curve.epsilons)]
        eps_at_best_alpha = normalized_curve.epsilon(best_alpha)
        d.append(
            dict(pmw_frac=pmw_fraction,
                 epsilon=epsilon,
                 best_alpha=best_alpha, 
                 best_vs_max=eps_at_best_alpha / normalized_curve.epsilon(64),
                 max_vs_5= normalized_curve.epsilon(64) / normalized_curve.epsilon(5),
                )
        )
df = pd.DataFrame(d)
px.line(df, title="Relative difference in tracking quality for best alpha vs alpha=64 vs alpha=5", range_y=[0,1.1],log_x=True, x="epsilon", y=["best_vs_max", "max_vs_5"], hover_data=["best_alpha"], facet_row="pmw_frac", height=600)

In [19]:
curves = {}
for i in range(5):
    # curves[f"Gaussian5-{i}"] =  GaussianCurve(sigma=5)
    curves[f"SVT_Pure5-{i}"] = PureDPtoRDP(1/10+1/10)
df = df_normalized_curves(curves)
px.area(df, x="alpha", y="normalized_rdp_epsilon", color="mech_type", line_group="mech_name")

In [20]:
curves = {}
for i in range(5):
    curves[f"Laplace5-{i}"] = LaplaceCurve(laplace_noise=5)
df = df_normalized_curves(curves)
px.area(df, x="alpha", y="normalized_rdp_epsilon", color="mech_type", line_group="mech_name")

In [21]:
curves = {}
for i in range(5):
    curves[f"Laplace5_PureDP-{i}"] = PureDPtoRDP(0.5/5)
df = df_normalized_curves(curves)
px.area(df, x="alpha", y="normalized_rdp_epsilon", color="mech_type", line_group="mech_name")